### Script summary: mapping satellite data for an area of interest (with optional .gif)
1. Choose area of interest; determine latitude and longitude subset using Google maps. 
2. Select good swaths of data using L3 data browser; days where you area of interest is not covered by clouds.
3. Load data in Jupyterlab as a xarray.Dataset. 
5. Create a plot where x is longitude, y is latitude, and the colormap is chlorophyll concentration.
6. Use the for loop to repeat plot over several days.
6. Optional: create .gif from .png files. 
6. Merge mulitple datasets and add time dimension for quantative analysis.
7. Manipulate to see trends across area and time of interest; take average, standard deviation, etc.

### 1. Import relevant packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmocean as cmo

# cartopy 
import cartopy.crs as ccrs       # ccrs contains information about projections
import cartopy    
import cartopy.feature as cfeature
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER

### 2. Construct the url where your data is located and define file names

In [2]:
base_url = 'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2021/' 
year_day = str(360)


filename = 'A2021' + year_day + '.L3m_DAY_CHL_chlor_a_9km.nc' # change year/file of interest here

url = base_url + year_day + '/' + filename

In [3]:
url

'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2021/360/A2021360.L3m_DAY_CHL_chlor_a_9km.nc'

### 3. Use a for loop to plot multiple days of chlorophyll data and save them as .png files

You input:
1. base_url
2. Day range between 001-3653
3. Year and file type under filename (ex. for 2021: 'A2021' and '.L3m_DAY_CHL_chlor_a_9km.nc')
4. Geographic region of interest by changing latitude and longitude maximum and minimum
5. Folder you want to save your figures in 

In [10]:
# produces individual .pngs for each day, which can later be combined into a .gif
base_url = 'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2021/' 

for day in range(133,139):
    year_day = str(day)
# this section constructs a filename from the filename on the NASA website
    filename = 'A2021' + year_day + '.L3m_DAY_CHL_chlor_a_9km.nc'
    time  = filename[1:5] + year_day
    url = base_url + year_day + '/' + filename
    modis = xr.open_dataset(url)
    subset = modis.where((modis.lon>=-75)&(modis.lon<-69)& (modis.lat>=-38)&(modis.lat<45), drop=True)
    fig = plt.figure(figsize=(10,10))
    regional_extent = [-75, -69, 38, 45]
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=-160))
    ax.set_extent(regional_extent)

# adds land, river, state border, and gridline features to map
    ax.add_feature(cartopy.feature.LAND,edgecolor='black',zorder=1) 
    ax.add_feature(cartopy.feature.RIVERS,zorder=2)

    states = cfeature.NaturalEarthFeature(category='cultural',
                                          name='admin_1_states_provinces_lines',
                                          scale='50m',
                                          facecolor='none'
    )
    ax.add_feature(states, edgecolor='gray', zorder=3)

    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True, 
                      dms=True, 
                      x_inline=False, 
                      y_inline=False
    )
    gl.top_labels = False
    gl.right_labels = False

    # plots the chlorophyll files
    im = subset.chlor_a.plot.pcolormesh(x='lon',
                                        y='lat', 
                                        norm=colors.LogNorm(vmin=0.5,vmax=8),
                                        cmap='Spectral_r',
                                        transform=ccrs.PlateCarree(),
    )
# prints day as cell runs, can change file name and save location here    
    # plt.title(time)
    print(year_day)
    figname = time + '.png'
    fig.savefig('figures2021/'+ figname, bbox_inches='tight', dpi=150) # saves figures in a different folder
    plt.close()

133
134
135
136
137
138


### 4. Merge into one dataset and merge time period of interest for quantative analysis
Averaging over a period of time can be useful in several scenarios. In this example, we will be comparing spring blooms in 2020 and 2021 on the Atlantic shelf. However, it can also be useful to see the general range of chlorophyll concentrations in a particular area, especially if that area has poor coverage via satellites. 

You input:
1. base_url 
2. daymin and daymax (ex. range 1-7, daymin=1, daymax=8)
2. Year and file type under filename (ex. for 2021: 'A2021' and '.L3m_DAY_CHL_chlor_a_9km.nc')
4. Geographic region of interest by changing latitude and longitude maximum and minimum

In [11]:
# merges the seperate days into one dataset
base_url = 'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2021/'

daymin=133
daymax=138

# naming protocal requires 0 and 00 for numbers in ones and tens
# for loop adds 0 and 00 
for day in range(daymin,daymax):

    if day<10:
        year_day = '00' + str(day) 
    elif day<100:
        year_day = '0' + str(day)
    else:
        year_day = str(day)
        
    filename = 'A2021' + year_day + '.L3m_DAY_CHL_chlor_a_9km.nc'

# subset_modis resamples based on geographic region of interest               
    time  = filename[1:5] + year_day
    year_day
    url = base_url + year_day + '/' + filename
    modis = xr.open_dataset(url)
    subset_modis = modis.where( (modis.lon>=-75)&
                                (modis.lon<-69)& 
                                (modis.lat>=38)&
                                (modis.lat<45), 
                                drop=True
    )
# adds time as a dimension using year_day naming protocol    
    subset_modis['time'] = int(time)
    subset_modis = subset_modis.assign_coords({'time': int(time)}).expand_dims('time')
# merges modis_time (which changes depending on which day the for loop is on) with subset_modis
# this fills in the time dimension
    if day == daymin:
        modis_time = subset_modis
    else:
        modis_time = xr.concat([modis_time,subset_modis],dim='time')

You input:
1. Geographic region of interest by changing latitude and longitude maximum and minimum
2. Name for new time averaged dataset 

In [12]:
# plots the average of your period of days
fig = plt.figure(figsize=(10,10))
regional_extent = [-75, -69, 38, 45]
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=-160))
ax.set_extent(regional_extent)


ax.add_feature(cartopy.feature.LAND,edgecolor='black',zorder=1) 
ax.add_feature(cartopy.feature.RIVERS,zorder=2)

states = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none'
)
ax.add_feature(states, edgecolor='gray', zorder=3)

gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True, 
                  dms=True, 
                  x_inline=False, 
                  y_inline=False
)
gl.top_labels = False
gl.right_labels = False
# resamples modis_time as an average with respect to the time dimension
modis_time_avg2021=modis_time.mean(dim=('time'))
modis_time_avg2021.chlor_a.plot.pcolormesh(x='lon',
                                y='lat', 
                                norm=colors.LogNorm(vmin=0.5,vmax=8),
                                cmap='Spectral_r',
                                transform=ccrs.PlateCarree(),)
fig.savefig('2021avg.png', bbox_inches='tight', dpi=150)
plt.close()

### 5. Optional: create a .gif for a period of time 
1. Open a terminal and go to the directory where the folder with your .png chlorophyll map files are. 
2. Copy and run the following line of code, which creates the .gif and saves it in the same folder as your .png files:

convert -delay 50 -loop 0 *.png animation.gif

Note: the number after delay determines how much time elapses between figures, and you can change the name of your .gif file. 

3. Open the .gif file in Jupyterlab to view it. 
4. Optional: go to https://cloudconvert.com/ to convert your .gif file into another type of video file, such as .mp4

### 6. Example: analysis of another year

You input:
1. base_url
2. Day range between 001-3653
3. Year and file type under filename (ex. for 2021: 'A2021' and '.L3m_DAY_CHL_chlor_a_9km.nc')
4. Geographic region of interest by changing latitude and longitude maximum and minimum
5. Folder you want to save your figures in 

In [13]:
# 2020: produces individual .pngs for each day, which can later be combined into a .gif
base_url = 'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2020/'
year_day = str(360)


filename = 'A2020' + year_day + '.L3m_DAY_CHL_chlor_a_9km.nc'

url = base_url + year_day + '/' + filename

for day in range(100,116):
    year_day = str(day)

    filename = 'A2020' + year_day + '.L3m_DAY_CHL_chlor_a_9km.nc'
    time  = filename[1:5] + year_day
    url = base_url + year_day + '/' + filename
    modis = xr.open_dataset(url)
    subset = modis.where((modis.lon>=-75)&(modis.lon<-69)& (modis.lat>=-38)&(modis.lat<45), drop=True)
    fig = plt.figure(figsize=(10,10))
    regional_extent = [-75, -69, 38, 45]
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=-160))
    ax.set_extent(regional_extent)


    ax.add_feature(cartopy.feature.LAND,edgecolor='black',zorder=1) 
    ax.add_feature(cartopy.feature.RIVERS,zorder=2)

    states = cfeature.NaturalEarthFeature(category='cultural',
                                          name='admin_1_states_provinces_lines',
                                          scale='50m',
                                          facecolor='none'
    )
    ax.add_feature(states, edgecolor='gray', zorder=3)

    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True, 
                      dms=True, 
                      x_inline=False, 
                      y_inline=False
    )
    gl.top_labels = False
    gl.right_labels = False

    # plots the chlorophyll files
    im = subset.chlor_a.plot.pcolormesh(x='lon',
                                        y='lat', 
                                        norm=colors.LogNorm(vmin=0.5,vmax=8),
                                        cmap='Spectral_r',
                                        transform=ccrs.PlateCarree(),
    )
    # plt.title(time)
    # print(year_day)
    figname = time + '.png'
    fig.savefig('figures2020/'+figname, bbox_inches='tight', dpi=150) # saves figures in a different folder
    plt.close()

You input:
1. base_url 
2. daymin and daymax (ex. range 1-7, daymin=1, daymax=8)
2. Year and file type under filename (ex. for 2021: 'A2021' and '.L3m_DAY_CHL_chlor_a_9km.nc')
4. Geographic region of interest by changing latitude and longitude maximum and minimum

In [14]:
# merges the seperate days into one dataset for a different year
base_url = 'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2021/'

daymin=133
daymax=138

for day in range(daymin,daymax):

    if day<10:
        year_day = '00' + str(day) 
    elif day<100:
        year_day = '0' + str(day)
    else:
        year_day = str(day)
        
    filename = 'A2021' + year_day + '.L3m_DAY_CHL_chlor_a_9km.nc'
        
        
    time  = filename[1:5] + year_day
    year_day
    url = base_url + year_day + '/' + filename
    modis = xr.open_dataset(url)
    subset_modis = modis.where( (modis.lon>=-75)&
                                (modis.lon<-69)& 
                                (modis.lat>=38)&
                                (modis.lat<45), 
                                drop=True
    )
    
    subset_modis['time'] = int(time)
    subset_modis = subset_modis.assign_coords({'time': int(time)}).expand_dims('time')
    
    if day == daymin:
        modis_time = subset_modis
    else:
        modis_time = xr.concat([modis_time,subset_modis],dim='time')

You input:
1. Geographic region of interest by changing latitude and longitude maximum and minimum
2. Name for new time averaged dataset 

In [15]:
# plots the yearly average for 2020
fig = plt.figure(figsize=(10,10))
regional_extent = [-75, -69, 38, 45]
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=-160))
ax.set_extent(regional_extent)


ax.add_feature(cartopy.feature.LAND,edgecolor='black',zorder=1) 
ax.add_feature(cartopy.feature.RIVERS,zorder=2)

states = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none'
)
ax.add_feature(states, edgecolor='gray', zorder=3)

gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True, 
                  dms=True, 
                  x_inline=False, 
                  y_inline=False
)
gl.top_labels = False
gl.right_labels = False

modis_time_avg2020=modis_time.mean(dim=('time'))
modis_time_avg2020.chlor_a.plot.pcolormesh(x='lon',
                                y='lat', 
                                norm=colors.LogNorm(vmin=0.5,vmax=8),
                                cmap='Spectral_r',
                                transform=ccrs.PlateCarree(),)
fig.savefig('2020avg.png', bbox_inches='tight', dpi=150)
plt.close()

### 7. Standard deviation of chlorophyll concentration across two time periods

You input:
1. The name of modis dataset from time period #1 and the name of modis dataset from time period #2
2. Define new dataset to denote standard deviation

In [16]:
#standard deviation between the two yearly averages 
modis2020_2021 = xr.merge([modis_time_avg2021,modis_time_avg2020])
std_modis2020_2021 = modis2020_2021.std()
std_modis2020_2021

<xarray.Dataset>
Dimensions:  ()
Data variables:
    palette  float64 48.77
    chlor_a  float32 3.597

This is the standard deviation of chlorophyll a concentration [mg/m^3] between the average datasets during the 2020 and 2021 spring blooms. 